In [ ]:
import json
import boto3
import uuid
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from io import StringIO

def convertToVertical(stocks, stock_industry_map, daily_3y_df):
    ticket_arr = []
    for name in stocks:
        temp_df = daily_3y_df[name].copy()
        temp_df['ticket'] = name
        temp_df['sector'] = stock_industry_map[name]
        ticket_arr.append(temp_df)
    vertical_df = pd.concat(ticket_arr)
    return vertical_df

def fetch_latest_daily_stocks():
    stocks = ['SPOT', 'AMD', 'HIG', 'ALL', 'SHEL', 'XOM', 'UPS', 'HON', 'PFE', 'UNH']

    stock_industry_map = {'SPOT':'Technology', 'AMD':'Technology', 
                      'HIG':'Financial Services', 'ALL':'Financial Services', 
                      'SHEL':'Energy', 'XOM':'Energy', 
                      'UPS':'Industry', 'HON':'Industry', 
                      'PFE':'Healthcare', 'UNH':'Healthcare'}
    
    list_tickers = yf.Tickers(stocks)

    daily_3y_df = list_tickers.download(period='1d', group_by='ticker')
    
    vertical_df = convertToVertical(stocks, stock_industry_map, daily_3y_df)

    vertical_df.reset_index(inplace = True)
    
    vertical_df.rename(columns={'Date':'trade_date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume', 
                            'Dividends':'dividends', 'Stock Splits':'stock_splits', 'ticket':'ticket'}, inplace = True)

    print(vertical_df.shape)
    
    vertical_df['trade_date'] = vertical_df['trade_date'].apply(lambda x: x.strftime('%Y-%m-%d')).astype(str)
    
    return vertical_df.to_json()

def lambda_handler(event, context):
    
    stock_json = fetch_latest_daily_stocks();
    
    client = boto3.client('kinesis')
    
    response = client.put_record(
        StreamName = 'birds',
        #Data = b'{"id":"200021", "name":"falcon"}',
        Data = stock_json,
        PartitionKey = str(uuid.uuid4())
        
    )
    
    print(response)
